Часть кода взята из https://github.com/SarangDeshmukh7/Recommendation-Engine/blob/master/Content_Based_Filtering.ipynb

# Коллаборативная модель
Модель предсказывает какую оценку пользователь поставит фильму, которого он еще не видел. Полученная метрика RMSE равна 0.7783

In [29]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np 
import pandas as pd 
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings;
warnings.filterwarnings('ignore')

Чтение из файла и создание тренировочного и тестового сета

In [2]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
data = Dataset.load_from_file('rating.csv', reader)

In [3]:
trainset, testset = train_test_split(data, test_size=0.05)

In [4]:
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.7783


0.7783432170839514

# Контентная модель
Модель предсказывает похожие фильмы. Метрики нет по причине, объясненной ранее

In [6]:
df_movies = pd.read_csv('movie.csv')
df_ratings = pd.read_csv('rating.csv')

In [7]:
df = pd.merge(df_movies,df_ratings)
df.dropna(inplace=True)
df['movieId']= df['movieId'].astype(str)
users = df["userId"].unique().tolist()

In [15]:
#list to capture watch history of the users
watch_train = []

# populate the list with the movie ID
for i in tqdm(users):
    temp = df[df["userId"] == i]["movieId"].tolist()
    watch_train.append(temp)

100%|████████████████████████████████████████████████████████████████████████| 138493/138493 [1:00:25<00:00, 38.20it/s]


In [16]:
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, 
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(watch_train, progress_per=200)

model.train(watch_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(199402934, 200002630)

In [17]:
watch = df[["movieId", "title"]]

# remove duplicates
watch.drop_duplicates(inplace=True, subset='movieId', keep="last")

# create movie id and tittle dictionary
watch_dict = watch.groupby('movieId')['title'].apply(list).to_dict()

In [22]:
def similar_watch(v, n = 6):
    
    # extract most similar movies for the input vector
    ms = model.wv.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar movies
    new_ms = []
    for j in ms:
        pair = (watch_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

Пример работы модели на фильме "Железный человек 3"

In [45]:
print(watch_dict['102125'])
similar_watch(model.wv['102125'])

['Iron Man 3 (2013)']


[('Star Trek Into Darkness (2013)', 0.9243887066841125),
 ('Man of Steel (2013)', 0.9156004786491394),
 ('Oblivion (2013)', 0.9052060842514038),
 ('Thor: Tales of Asgard (2011)', 0.8952834010124207),
 ('Now You See Me (2013)', 0.8940366506576538),
 ('Iron Man: Rise Of Technovore (2013)', 0.8910979628562927)]

# Гибридная модель
Сначала для пользователя подбирается рейтинг фильмов, которым он мог бы поставить максимальный балл, затем к каждому из этих фильмов добавляются похожие на него и выводится общий список

In [50]:
movie_ids = df["movieId"].unique().tolist()

In [109]:
def return_top_movies(Id, k1, k2):
    """
    Args:
        k1(int) - number of top movies from collaborative model
        k2(int) - number of top movies from content-based model

    Returns:
    A list of k1*k2 recommended movies with similarity score
    """

    def return_second_value_of_set(some_set):
        return some_set[1]
    lst = []
    lst_rating = []
    for i in movie_ids:
        lst_rating.append((i,algo.predict(str(Id), i)[3]))    
    lst_rating = sorted(lst_rating, key=return_second_value_of_set, reverse=True)[:k1]    
    for i in collaborative_top:
        lst.append(similar_watch(model.wv[i[0]], k2))
    return lst

Пример для пользователя с Id=5, три фильма выбраны из коллаборативной модели, затем к каждому из этих 3-х фильмов добавляются по три похожих фильма. Фильмы из коллаборативной модели не выводятся

In [110]:
return_top_movies(Id=5, k1=3, k2=3)

[[('Pianist, The (2002)', 0.6864071488380432),
  ('My Neighbor Totoro (Tonari no Totoro) (1988)', 0.6775856614112854),
  ('Monty Python Live at the Hollywood Bowl (1982)', 0.6604493856430054)],
 [('Prime Suspect 2 (1992)', 0.9227423071861267),
  ('Hit, The (1984)', 0.918782114982605),
  ('Whales of August, The (1987)', 0.9100519418716431)],
 [('Gladiator (1992)', 0.6853943467140198),
  ('Band of Brothers (2001)', 0.6683748960494995),
  ('Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984)',
   0.6599408388137817)]]